In [1]:
# %reset

In [2]:
import pandas as pd
import collections
import pickle

## Assess

Not Tidy  
tweet_id(s) not common across pandas DataFrames  
preparation for eventual merge, with real time tweepy.API Twitter Data Query, the 3rd pandas DataFrame
* Twitter Data Query is a tweet_id driven process  
* tweet_id(s) __common__ to both image_predictions_df_clean and twitter_archive_enhanced_clean_df will drive the Twitter Data Query
* common tweet_id(s) required for eventual tweet_id driven merge of all 3 pandas DataFrames

In [3]:
# read cleaned - twitter_archive_enhanced_clean_df 
with open('twitter_archive_enhanced_clean_df.pkl', 'rb') as f:
    twitter_archive_enhanced_clean_df = pickle.load(f) 

# make sure we can read it 
print('twitter_archive_enhanced_clean_df.shape')
print(twitter_archive_enhanced_clean_df.shape)

twitter_archive_enhanced_clean_df.shape
(2163, 11)


In [4]:
# read cleaned - image_predictions_df_clean
with open('image_predictions_df_clean.pkl', 'rb') as f:
    image_predictions_df_clean = pickle.load(f)
    
# make sure we can read it
print('image_predictions_df_clean.shape')
print(image_predictions_df_clean.shape)

image_predictions_df_clean.shape
(2072, 12)


tweet_id(s) NOT common - different row counts

look for tweet_id(s) in both DataFrames method 1 of 2

In [5]:
# Eventually there will be 3 pandas DataFrames merged. 
# The merges will be based on the tweet_id pandas Series, column 
# in each of the 3 pandas DataFrames. 
# Identify common tweet_id(s) method 1 of 2. 
# Using 2 methods may be seen as redundant it can also be 
# seen thorough. Method 1 of 2 iterates through the  
# image_predictions_df_clean pandas DataFrame and finds tweet_id(s) 
# common with the twitter_archive_enhanced_clean_df pandas DataFrame.

image_only_count = 0
image_only_list = []
image_driven_common_list = []

twitter_and_image_count = 0

for index, row in image_predictions_df_clean.iterrows():
    
    # (df.index == 'entry').any()
    tweet_id_is_common = \
    (twitter_archive_enhanced_clean_df.tweet_id == row.tweet_id).any()
    # print('tweet_id_is_common - {}'.format(tweet_id_is_common))
    # print('type(tweet_id_is_common) - {}\n'.format(type(tweet_id_is_common)))

    if tweet_id_is_common :
        twitter_and_image_count += 1
        image_driven_common_list.append(row.tweet_id)

    if not tweet_id_is_common :
        image_only_count += 1
        image_only_list.append(row.tweet_id)
        
print('twitter_and_image_count - {}'.format(twitter_and_image_count))
print('len(image_driven_common_list) - {}\n'.format(len(image_driven_common_list)))

print('image_only_count - {}'.format(image_only_count))
print('len(image_only_list) - {}'.format(len(image_only_list)))
 

twitter_and_image_count - 1982
len(image_driven_common_list) - 1982

image_only_count - 90
len(image_only_list) - 90


look for tweet_id(s) in both DataFrames method 2 of 2

In [6]:
# method 2 of 2 iterate through 
# twitter_archive_enhanced_clean_df pandas DataFrame
# find tweet_id(s) common with the 
# image_predictions_df_clean pandas DataFrame

twitter_only_count = 0
twitter_only_list = []
twitter_driven_common_list = []

twitter_and_image_count = 0
 
for index, row in twitter_archive_enhanced_clean_df.iterrows():
         
    tweet_id_is_common = (image_predictions_df_clean.tweet_id == row.tweet_id).any()    
    # print('x - {}'.format(x))
    # print('type(x) - {}\n'.format(type(x)))

    if tweet_id_is_common :
        twitter_and_image_count += 1
        twitter_driven_common_list.append(row.tweet_id)

    if not tweet_id_is_common :
        twitter_only_count += 1
        twitter_only_list.append(row.tweet_id)
        
print('twitter_and_image_count - {}'.format(twitter_and_image_count))
print('len(twitter_driven_common_list) - {}\n'.format(len(twitter_driven_common_list)))

print('twitter_only_count - {}'.format(twitter_only_count))
print('len(twitter_only_list) - {}\n'.format(len(twitter_only_list)))
    

twitter_and_image_count - 1982
len(twitter_driven_common_list) - 1982

twitter_only_count - 181
len(twitter_only_list) - 181



verify common tweet_id(s) arrived at 2 ways are indeed commom

In [7]:
# confirm common tweet_id derived lists match
collections.Counter(image_driven_common_list) == \
collections.Counter(twitter_driven_common_list)

True

## Clean

### Define

synchronize image and twitter archive DataFrames - only rows with common tweet_id(s)

### Code

In [8]:
twitter_archive_common_df = \
twitter_archive_enhanced_clean_df\
[twitter_archive_enhanced_clean_df.tweet_id.isin\
(twitter_driven_common_list)]

print('twitter_archive_common_df.shape')
print(twitter_archive_common_df.shape)


twitter_archive_common_df.shape
(1982, 11)


In [9]:
image_common_df = image_predictions_df_clean\
[image_predictions_df_clean.tweet_id.isin(image_driven_common_list)]

print('image_common_df.shape')
print(image_common_df.shape)

image_common_df.shape
(1982, 12)


## Test

verify tweet_id(s) are indeed common in the 2 DataFrames

In [10]:
# iterate through image_common_df
# test - veryify image_common_df - twitter_archive_common_df 
# tweet_id match
 
image_twitter_common_tweet_id_counter = 0

print('image_common_df.shape')
print(image_common_df.shape)

print('twitter_archive_common_df.shape')
print(twitter_archive_common_df.shape)
print()

for index, row in image_common_df.iterrows():
    
    tweet_id_is_common = \
    (twitter_archive_common_df.tweet_id == row.tweet_id).any() 
    
    if tweet_id_is_common :
        image_twitter_common_tweet_id_counter +=1
    else:
        raise Exception('tweet_id common error - image tweet_id is: {}'.format(row.tweet_id))
    
print('image_twitter_common_tweet_id_counter - {}'.\
      format(image_twitter_common_tweet_id_counter))


image_common_df.shape
(1982, 12)
twitter_archive_common_df.shape
(1982, 11)

image_twitter_common_tweet_id_counter - 1982


Successful pandas DataFrame merge preparation -  
pandas DataFrame - image_common_df  
and  
pandas DataFrame - twitter_archive_common_df  
have the SAME tweet_id(s)  

In [11]:
with open('image_common_df.pkl', 'wb') as f:
        pickle.dump(image_common_df, f)

In [12]:
# make sure we can read it back in 
with open('image_common_df.pkl', 'rb') as f:
    image_common_df = pickle.load(f)
    
print('image_common_df.shape')
print(image_common_df.shape)

image_common_df.shape
(1982, 12)


In [13]:
with open('twitter_archive_common_df.pkl', 'wb') as f:
        pickle.dump(twitter_archive_common_df, f)

In [14]:
# make sure we can read it back in 

with open('twitter_archive_common_df.pkl', 'rb') as f:
    twitter_archive_common_df = pickle.load(f)
    
print('twitter_archive_common_df.shape')
print(twitter_archive_common_df.shape)

twitter_archive_common_df.shape
(1982, 11)
